# Notebook 26 — Opening Recommender Model: Training Pipeline

### 0. Overview and Goals

This notebook defines the full pipeline for training the chess opening recommender model.  
The objective is to predict **player–opening performance scores** ((wins + (0.5 * draws) / num games)) for openings a player hasn’t yet played, based on their results in the openings they *have* played.  

The model will use **matrix factorization** with **stochastic gradient descent (SGD)** to learn latent factors representing player and opening characteristics.  
All computations will be implemented in **PyTorch**, with data loaded from my local **DuckDB** database.

**High-level specs:**
- Use only *White* openings initially (we’ll extend to Black later).  
- Data source: processed player–opening stats from local DuckDB.  
- Predict: normalized “score” = win rate ((wins + 0.5 x draws) / total games).  
- Filter: only include entries with ≥ `MIN_GAMES_THRESHOLD` (default = 50).  
- Ignore: rating differences, time controls, and other metadata.  
- Model parameters (to be defined in appropriate places for easy editing):  
  - `NUM_FACTORS`, `LEARNING_RATE`, `BATCH_SIZE`, `N_EPOCHS`, `NUM_PLAYERS_TO_PROCESS`  
- Logging and checkpoints throughout for reproducibility.  
- All random operations seeded for deterministic runs.  

---

### 1. Data Extraction
- Connect to local DuckDB
- Pull all processed player–opening statistics from
- Verify schema consistency:  
  - Required columns: `player_id`, `opening_id`, `eco`, `num_games`, `wins`, `draws`, `losses`.  
- Include a row-count sanity check.

---

### 2. Data Sanitization & Normalization
- Optionally normalize scores if needed for MF convergence.  
- Drop players with no qualifying openings and openings with no qualifying players.  
  - I believe there shouldn't be any but we'll double check.
- Resequence player_id and opening_id to be sequential integers - right now there are gaps because of entries we deleted from the DB 
- Check for sparsity consistency (no implicit zeros yet).  
- Note that this data has already been split in to white and black games further up the pipeline

### Data Quality
- Drop entries with fewer than `MIN_GAMES_THRESHOLD` games
- Handle any duplicate `(player_id, opening_id)` combinations
- Remove players with no qualifying openings
- Remove openings with no qualifying players
- Verify no null values remain

### ECO Codes
- Keep ECO codes for later categorical encoding (Step 4)
- ECO will be used as opening side information (similar to rating for players)

### Confidence Weighting
- Use `MIN_GAMES_THRESHOLD = 10` to keep more data
- Add a **confidence weight** column: `confidence = num_games / (num_games + K)` where K ≈ 50
- This weight will be used in the loss function to down-weight uncertain predictions
- High-game-count entries → high confidence → larger loss impact
- Low-game-count entries → low confidence → smaller loss impact

### Player Rating (Side Information)
- **Player ratings are side information** - they describe player characteristics, not individual player-opening interactions
- Ratings will be stored separately and joined to player embeddings during training
- We'll **normalize ratings** (likely z-score normalization) to avoid scaling issues with the embedding layer
- Rating normalization will be done once after extraction, not per-row

---

### 3. Data Splits
- Split into train/test/val sets.  
- Ensure every player and every opening appears at least once in the training data.  
- Strategy:  
  - Sample unique players and openings to guarantee coverage in train.  
  - Remaining data → stratified random split into train/test.  
  - Deduplicate and merge unique IDs back into train if needed.

---

### 4. Enumerate Categorical Variables
- Enumerate `eco` (if included) as an integer categorical variable.  
- Confirm all columns are numeric and compatible with PyTorch tensors.  
- Verify no missing or out-of-range IDs.

---

### 5. Training Data Structure
- Each row: one `(player_id, opening_id, score)` record.
- Include other fields- eco, num games etc
- Convert DataFrame to PyTorch tensors (`torch.long` for IDs, `torch.float` for scores).  
- Log dataset shapes and sparsity metrics.

---

### 6. Training Setup
Define constants:
- `LEARNING_RATE`, `BATCH_SIZE`, `N_EPOCHS`, `NUM_FACTORS`  
- Loss functions: MSE and RMSE  
- Activation: sigmoid or none (depending on score normalization)  
- Optimizer: SGD  
- Figure out if there's anything else we need to design or specify

Implement helper functions:
- `train_one_epoch()`
- `evaluate_model()`
- `calculate_rmse()`
- `save_checkpoint()`  

Ensure detailed logging, ETA reporting, and reproducible random seeds.

---

### 7. Training Loop
- Initialize player and opening embeddings.  
- Iterate through epochs with mini-batch SGD (`BATCH_SIZE = 1024`).  
- Compute and log MSE/RMSE per epoch.  
- Save model checkpoints locally after each epoch.

---

### 8. Evaluation
- Evaluate on test set.  
- Report MSE, RMSE, and visual diagnostics (predicted vs actual score).  
- Inspect a few player and opening latent factors for sanity.

---

### 9. Cross-Validation & Hyperparameter Tuning
- Define ranges for:  
  - `NUM_FACTORS`, `LEARNING_RATE`, `BATCH_SIZE`, `N_EPOCHS`  
- Perform small-scale grid or random search for best configuration.  
- Compare validation RMSE across runs.

---

### 10. Next Steps
- Extend model to include Black openings.  
- Experiment with hybrid inputs (player rating, ECO grouping).  
- Consider implicit feedback handling (unplayed openings as zeros).  
- Integrate trained model into API for recommendation output.

---

**Notes:**  
- Every random seed and parameter definition will be explicit.  
- Every major step includes row-count, schema, and type validation.  
- Model artifacts and logs will be saved locally for reproducibility.


## Step 1: Data Extraction

Connect to DuckDB and extract all player-opening statistics.
Verify schema and perform sanity checks.

In [1]:
# Setup and imports
from pathlib import Path
import pandas as pd
import sys

# Add utils to path
sys.path.append(str(Path.cwd() / 'utils'))
from database.db_utils import get_db_connection

# Configuration
DB_PATH = Path.cwd().parent / "data" / "processed" / "chess_games.db"
COLOR_FILTER = 'w'  # 'w' for white, 'b' for black

print("=" * 60)
print("STEP 1: DATA EXTRACTION")
print("=" * 60)
print(f"\n📁 Database: {DB_PATH}")
print(f"📁 Database exists: {DB_PATH.exists()}")
print(f"🎨 Color filter: {'White' if COLOR_FILTER == 'w' else 'Black'}")

if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found at {DB_PATH}")

STEP 1: DATA EXTRACTION

📁 Database: /Users/a/Documents/personalprojects/chess-opening-recommender/data/processed/chess_games.db
📁 Database exists: True
🎨 Color filter: White


In [2]:
# Connect to DuckDB and extract player-opening statistics
con = get_db_connection(str(DB_PATH))

try:
    print(f"\n1️⃣  Extracting player-opening statistics (color: '{COLOR_FILTER}')...")
    
    # Extract stats with calculated score and num_games
    # Filter by color and calculate score in the database
    query = f"""
        SELECT 
            pos.player_id,
            pos.opening_id,
            pos.num_wins + pos.num_draws + pos.num_losses as num_games,
            (pos.num_wins + (pos.num_draws * 0.5)) / 
                NULLIF(pos.num_wins + pos.num_draws + pos.num_losses, 0) as score,
            o.eco
        FROM player_opening_stats pos
        JOIN opening o ON pos.opening_id = o.id
        WHERE pos.color = '{COLOR_FILTER}'
        ORDER BY pos.player_id, pos.opening_id
    """
    
    raw_data = pd.DataFrame(con.execute(query).df())
    
    print(f"   ✓ Extracted {len(raw_data):,} rows")
    
    # Schema verification
    print("\n2️⃣  Verifying schema...")
    required_columns = ['player_id', 'opening_id', 'num_games', 'score', 'eco']
    
    for col in required_columns:
        if col not in raw_data.columns:
            raise ValueError(f"Missing required column: {col}")
    
    print(f"   ✓ All required columns present: {required_columns}")
    
    # Data types verification
    print("\n3️⃣  Checking data types...")
    print(f"   • player_id: {raw_data['player_id'].dtype}")
    print(f"   • opening_id: {raw_data['opening_id'].dtype}")
    print(f"   • num_games: {raw_data['num_games'].dtype}")
    print(f"   • score: {raw_data['score'].dtype}")
    print(f"   • eco: {raw_data['eco'].dtype}")
    
    # Basic statistics
    print("\n4️⃣  Data statistics...")
    print(f"   • Total rows: {len(raw_data):,}")
    print(f"   • Unique players: {raw_data['player_id'].nunique():,}")
    print(f"   • Unique openings: {raw_data['opening_id'].nunique():,}")
    print(f"   • Total games (sum): {raw_data['num_games'].sum():,}")
    
    # Player ID range
    print(f"\n   Player ID range:")
    print(f"   • Min: {raw_data['player_id'].min()}")
    print(f"   • Max: {raw_data['player_id'].max()}")
    
    # Opening ID range
    print(f"\n   Opening ID range:")
    print(f"   • Min: {raw_data['opening_id'].min()}")
    print(f"   • Max: {raw_data['opening_id'].max()}")
    
    # Games per entry statistics
    print(f"\n   Games per entry:")
    print(f"   • Min: {raw_data['num_games'].min()}")
    print(f"   • Max: {raw_data['num_games'].max()}")
    print(f"   • Mean: {raw_data['num_games'].mean():.1f}")
    print(f"   • Median: {raw_data['num_games'].median():.0f}")
    
    # Score statistics
    print(f"\n   Score distribution:")
    print(f"   • Min: {raw_data['score'].min():.4f}")
    print(f"   • Max: {raw_data['score'].max():.4f}")
    print(f"   • Mean: {raw_data['score'].mean():.4f}")
    print(f"   • Median: {raw_data['score'].median():.4f}")
    
    # Check for null values
    print("\n5️⃣  Checking for null values...")
    null_counts = raw_data.isnull().sum()
    if null_counts.sum() == 0:
        print("   ✓ No null values found")
    else:
        print("   ⚠️  Found null values:")
        for col, count in null_counts[null_counts > 0].items():
            print(f"      • {col}: {count} nulls")
    
    # Sample data
    print("\n6️⃣  Sample of extracted data (first 10 rows):")
    print(raw_data.head(10).to_string())
    
    print("\n" + "=" * 60)
    print("✅ DATA EXTRACTION COMPLETE")
    print("=" * 60)
    print(f"\nData shape: {raw_data.shape}")
    print(f"Columns: {list(raw_data.columns)}")
    
finally:
    con.close()
    print("\n✓ Database connection closed")


1️⃣  Extracting player-opening statistics (color: 'w')...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   ✓ Extracted 11,877,700 rows

2️⃣  Verifying schema...
   ✓ All required columns present: ['player_id', 'opening_id', 'num_games', 'score', 'eco']

3️⃣  Checking data types...
   • player_id: int32
   • opening_id: int32
   • num_games: int32
   • score: float64
   • eco: object

4️⃣  Data statistics...
   • Total rows: 11,877,700
   • Unique players: 49,906
   • Unique openings: 2,991
   • Total games (sum): 235,152,459

   Player ID range:
   • Min: 1
   • Max: 50000

   Opening ID range:
   • Min: 2
   • Max: 3589

   Games per entry:
   • Min: 1
   • Max: 13462
   • Mean: 19.8
   • Median: 3

   Score distribution:
   • Min: 0.0000
   • Max: 1.0000
   • Mean: 0.5006
   • Median: 0.5000

5️⃣  Checking for null values...
   ✓ No null values found

6️⃣  Sample of extracted data (first 10 rows):
   player_id  opening_id  num_games     score  eco
0          1          39          6  0.666667  A00
1          1          49          2  0.500000  A00
2          1          53          1  0

## Step 2: Data Sanitization & Normalization

Filter low-quality data, handle duplicates, and prepare for training.

In [3]:
# 2a. Filter low-quality data, handle duplicates, and prepare for training.

import numpy as np

# Configuration
MIN_GAMES_THRESHOLD = 10

print("=" * 60)
print("STEP 2: DATA SANITIZATION & NORMALIZATION")
print("=" * 60)
print(f"\n⚙️  Configuration:")
print(f"   • MIN_GAMES_THRESHOLD: {MIN_GAMES_THRESHOLD}")

# Start with raw_data from Step 1
print(f"\n📊 Starting data shape: {raw_data.shape}")
print(f"   • Rows: {len(raw_data):,}")
print(f"   • Unique players: {raw_data['player_id'].nunique():,}")
print(f"   • Unique openings: {raw_data['opening_id'].nunique():,}")

# 1. Filter by minimum games threshold
print(f"\n1️⃣  Filtering entries with < {MIN_GAMES_THRESHOLD} games...")
before_filter = len(raw_data)
clean_data = raw_data.query(f'num_games >= {MIN_GAMES_THRESHOLD}').copy()
num_rows_after_filter = len(clean_data)
num_rows_filtered_out = before_filter - num_rows_after_filter

print(f"   • Before: {before_filter:,} rows")
print(f"   • After: {num_rows_after_filter:,} rows")
print(f"   • Filtered out: {num_rows_filtered_out:,} rows ({100*num_rows_filtered_out/before_filter:.1f}%)")

# 2. Check for duplicates
print(f"\n2️⃣  Checking for duplicate (player_id, opening_id) combinations...")
num_duplicates = clean_data.duplicated(subset=['player_id', 'opening_id']).sum()

if num_duplicates > 0:
    print(f"   ⚠️  Found {num_duplicates} duplicate entries")
    dup_mask = clean_data.duplicated(subset=['player_id', 'opening_id'], keep=False)
    print("\n   Sample of duplicates:")
    print(clean_data[dup_mask].head(10).to_string())
    
    # Keep only first occurrence of any duplicate player-opening pair
    print("\n   Removing duplicates (keeping first occurrence)...")
    clean_data = pd.DataFrame.drop_duplicates(clean_data, subset=['player_id', 'opening_id'], keep='first')
    print(f"   ✓ After deduplication: {len(clean_data):,} rows")
else:
    print(f"   ✓ No duplicates found")

# 3. Remove players with no qualifying openings
print(f"\n3️⃣  Removing players with no qualifying openings...") # Note that a few players only play stuff like the Van't Kruijs which we've excluded, so a small numer of players will be excluded here
players_before = clean_data['player_id'].nunique()

# Count openings per player
num_openings_per_player = pd.DataFrame(clean_data.groupby('player_id').size(), columns=['count'])
players_with_data = num_openings_per_player[num_openings_per_player['count'] > 0].index.tolist()

# Filter
clean_data = clean_data[clean_data['player_id'].isin(players_with_data)]
players_after = clean_data['player_id'].nunique()

print(f"   • Players before: {players_before:,}")
print(f"   • Players after: {players_after:,}")
print(f"   • Removed: {players_before - players_after}")

# 4. Remove openings with no qualifying players
print(f"\n4️⃣  Removing openings with no qualifying players...")
num_openings_before = clean_data['opening_id'].nunique()

# Use pd.DataFrame.groupby() to count players per opening
num_players_per_opening = pd.DataFrame(clean_data.groupby('opening_id').size(), columns=['count'])
openings_with_data = num_players_per_opening[num_players_per_opening['count'] > 0].index.tolist()

# Filter using pd.DataFrame.isin()
clean_data = clean_data[clean_data['opening_id'].isin(openings_with_data)]
openings_after = clean_data['opening_id'].nunique()

print(f"   • Openings before: {num_openings_before:,}")
print(f"   • Openings after: {openings_after:,}")
print(f"   • Removed: {num_openings_before - openings_after}")

# 5. Verify no null values using pd.isna()
print(f"\n5️⃣  Verifying no null values...")
null_counts = pd.DataFrame.isna(clean_data).sum()
if null_counts.sum() == 0:
    print("   ✓ No null values found")
else:
    print("   ⚠️  Found null values:")
    for col, count in null_counts[null_counts > 0].items():
        print(f"      • {col}: {count} nulls")
    # Drop rows with nulls using pd.DataFrame.dropna()
    clean_data = pd.DataFrame.dropna(clean_data)
    print(f"   ✓ Dropped null rows. New shape: {clean_data.shape}")

# TODO: Add confidence weighting column
# TODO: Extract and normalize player ratings (side information)

# Reset index using pd.DataFrame.reset_index()
clean_data = pd.DataFrame.reset_index(clean_data, drop=True)

# Final statistics using pd functions
print(f"\n6️⃣  Final data statistics:")
print(f"   • Total rows: {len(clean_data):,}")
print(f"   • Unique players: {pd.Series.nunique(clean_data['player_id']):,}")
print(f"   • Unique openings: {pd.Series.nunique(clean_data['opening_id']):,}")
print(f"   • Total games: {pd.Series.sum(clean_data['num_games']):,}")
print(f"   • Avg games per entry: {pd.Series.mean(clean_data['num_games']):.1f}")
print(f"   • Avg openings per player: {len(clean_data) / pd.Series.nunique(clean_data['player_id']):.1f}")
print(f"   • Avg players per opening: {len(clean_data) / pd.Series.nunique(clean_data['opening_id']):.1f}")

# Score distribution using pd functions
print(f"\n   Score statistics:")
print(f"   • Min: {pd.Series.min(clean_data['score']):.4f}")
print(f"   • 25th percentile: {pd.Series.quantile(clean_data['score'], 0.25):.4f}")
print(f"   • Median: {pd.Series.median(clean_data['score']):.4f}")
print(f"   • 75th percentile: {pd.Series.quantile(clean_data['score'], 0.75):.4f}")
print(f"   • Max: {pd.Series.max(clean_data['score']):.4f}")
print(f"   • Mean: {pd.Series.mean(clean_data['score']):.4f}")
print(f"   • Std: {pd.Series.std(clean_data['score']):.4f}")

# Sample of cleaned data using pd.DataFrame.sample()
print(f"\n7️⃣  Sample of cleaned data (10 random rows):")
print(pd.DataFrame.sample(clean_data, min(10, len(clean_data)), random_state=42).to_string())

print("\n" + "=" * 60)
print("✅ DATA SANITIZATION COMPLETE")
print("=" * 60)
print(f"\nCleaned data shape: {clean_data.shape}")
print(f"Data reduction: {100 * (1 - len(clean_data)/len(raw_data)):.1f}%")

STEP 2: DATA SANITIZATION & NORMALIZATION

⚙️  Configuration:
   • MIN_GAMES_THRESHOLD: 10

📊 Starting data shape: (11877700, 5)
   • Rows: 11,877,700
   • Unique players: 49,906
   • Unique openings: 2,991

1️⃣  Filtering entries with < 10 games...
   • Before: 11,877,700 rows
   • After: 2,975,888 rows
   • Filtered out: 8,901,812 rows (74.9%)

2️⃣  Checking for duplicate (player_id, opening_id) combinations...
   ✓ No duplicates found

3️⃣  Removing players with no qualifying openings...
   • Players before: 49,822
   • Players after: 49,822
   • Removed: 0

4️⃣  Removing openings with no qualifying players...
   • Openings before: 2,718
   • Openings after: 2,718
   • Removed: 0

5️⃣  Verifying no null values...
   ✓ No null values found

6️⃣  Final data statistics:
   • Total rows: 2,975,888
   • Unique players: 49,822
   • Unique openings: 2,718
   • Total games: 211,951,019
   • Avg games per entry: 71.2
   • Avg openings per player: 59.7
   • Avg players per opening: 1094.9

  

In [4]:
# 2b. Apply hierarchical Bayesian shrinkage to adjust scores based on sample size confidence

# A lot of our player-opening entries have a small number of games played, because openings are so specific.
# This introduces sample size issues.

# We'll use TWO-LEVEL shrinkage:
# Level 1: Calculate opening-specific means (these are our "ground truth" for each opening)
# Level 2: Shrink individual player-opening scores toward their opening's mean
# This is better than shrinking toward global mean because different openings have different baseline win rates

print("=" * 60)
print("STEP 2B: HIERARCHICAL BAYESIAN SCORE ADJUSTMENT")
print("=" * 60)

# Configuration for Bayesian shrinkage
K_PLAYER = 50  # Shrinkage constant for player-opening scores

print(f"\n⚙️  Configuration:")
print(f"   • K_PLAYER (shrinkage constant): {K_PLAYER}")
print(f"   • Method: Two-level empirical Bayes shrinkage")
print(f"   • Level 1: Calculate opening-specific means")
print(f"   • Level 2: Shrink player scores toward opening means")

# Calculate global mean score for comparison
global_mean_score = clean_data["score"].mean()
print(f"\n📊 Global statistics:")
print(f"   • Global mean score: {global_mean_score:.4f}")
print(f"   • Total entries: {len(clean_data):,}")
print(f"   • Unique openings: {clean_data['opening_id'].nunique():,}")

# Store original scores for comparison
clean_data = clean_data.copy()  # Best practice: work on a copy
clean_data["score_original"] = clean_data["score"].copy()

# LEVEL 1: Calculate opening-specific means and statistics
print(f"\n1️⃣  LEVEL 1: Calculating opening-specific means...")

opening_stats = (
    clean_data.groupby("opening_id")
    .agg(
        {
            "score": "mean",
            "num_games": "sum",
            "player_id": "count",  # Number of players who played this opening
        }
    )
    .rename(
        columns={
            "score": "opening_mean",
            "num_games": "opening_total_games",
            "player_id": "opening_num_players",
        }
    )
)

print(f"   ✓ Calculated means for {len(opening_stats):,} openings")

# Opening mean statistics
print(f"\n   Opening mean score distribution:")
print(f"   • Min: {opening_stats['opening_mean'].min():.4f}")
print(f"   • 25th percentile: {opening_stats['opening_mean'].quantile(0.25):.4f}")
print(f"   • Median: {opening_stats['opening_mean'].median():.4f}")
print(f"   • 75th percentile: {opening_stats['opening_mean'].quantile(0.75):.4f}")
print(f"   • Max: {opening_stats['opening_mean'].max():.4f}")
print(f"   • Std: {opening_stats['opening_mean'].std():.4f}")

# Show distribution of opening sizes
print(f"\n   Opening sample size distribution:")
print(
    f"   • Total games per opening (median): {opening_stats['opening_total_games'].median():.0f}"
)
print(
    f"   • Players per opening (median): {opening_stats['opening_num_players'].median():.0f}"
)
print(
    f"   • Total games range: [{opening_stats['opening_total_games'].min():.0f}, {opening_stats['opening_total_games'].max():.0f}]"
)
print(
    f"   • Players range: [{opening_stats['opening_num_players'].min():.0f}, {opening_stats['opening_num_players'].max():.0f}]"
)

# Merge opening means back into main dataframe
clean_data = clean_data.merge(
    opening_stats[["opening_mean"]], left_on="opening_id", right_index=True, how="left"
)

# LEVEL 2: Shrink player-opening scores toward opening-specific means
print(f"\n2️⃣  LEVEL 2: Shrinking player scores toward opening means...")
print(
    f"   Formula: adjusted_score = (num_games × player_score + {K_PLAYER} × opening_mean) / (num_games + {K_PLAYER})"
)

numerator = (clean_data["num_games"] * clean_data["score_original"]) + (
    K_PLAYER * clean_data["opening_mean"]
)
denominator = clean_data["num_games"] + K_PLAYER
clean_data["score"] = numerator / denominator

print(f"   ✓ Scores adjusted for {len(clean_data):,} entries")

# Calculate confidence weights (will be used in loss function later)
print(f"\n3️⃣  Calculating confidence weights...")
clean_data["confidence"] = clean_data["num_games"] / (
    clean_data["num_games"] + K_PLAYER
)
print(f"   ✓ Confidence weights calculated")
print(f"   • Formula: confidence = num_games / (num_games + {K_PLAYER})")
print(
    f"   • Range: [{clean_data['confidence'].min():.4f}, {clean_data['confidence'].max():.4f}]"
)

# Statistics on the adjustment
score_diff = clean_data["score"] - clean_data["score_original"]
print(f"\n4️⃣  Adjustment statistics:")
print(f"   • Mean adjustment: {score_diff.mean():.6f}")
print(f"   • Std adjustment: {score_diff.std():.6f}")
print(f"   • Max adjustment: {score_diff.max():.6f}")
print(f"   • Min adjustment: {score_diff.min():.6f}")

# Show distribution of adjustments
print(f"\n   Adjustment by num_games quartiles:")
quartiles = clean_data["num_games"].quantile([0.25, 0.5, 0.75])
print(
    f"   • 25th percentile (n={quartiles[0.25]:.0f} games): avg adjustment = {score_diff[clean_data['num_games'] <= quartiles[0.25]].mean():.6f}"
)
print(
    f"   • 50th percentile (n={quartiles[0.5]:.0f} games): avg adjustment = {score_diff[(clean_data['num_games'] > quartiles[0.25]) & (clean_data['num_games'] <= quartiles[0.5])].mean():.6f}"
)
print(
    f"   • 75th percentile (n={quartiles[0.75]:.0f} games): avg adjustment = {score_diff[(clean_data['num_games'] > quartiles[0.5]) & (clean_data['num_games'] <= quartiles[0.75])].mean():.6f}"
)
print(
    f"   • >75th percentile (n>{quartiles[0.75]:.0f} games): avg adjustment = {score_diff[clean_data['num_games'] > quartiles[0.75]].mean():.6f}"
)

# New score distribution after adjustment
print(f"\n5️⃣  Adjusted score statistics:")
print(f"   • Min: {clean_data['score'].min():.4f}")
print(f"   • 25th percentile: {clean_data['score'].quantile(0.25):.4f}")
print(f"   • Median: {clean_data['score'].median():.4f}")
print(f"   • 75th percentile: {clean_data['score'].quantile(0.75):.4f}")
print(f"   • Max: {clean_data['score'].max():.4f}")
print(f"   • Mean: {clean_data['score'].mean():.4f}")
print(f"   • Std: {clean_data['score'].std():.4f}")

# Detailed sample showing the effect across different game counts
print(f"\n6️⃣  Sample comparisons (showing effect of hierarchical shrinkage):")
print(f"\n   {'='*120}")
print(f"   Low-game entries (10-20 games) - HIGH shrinkage toward opening mean:")
print(f"   {'='*120}")

low_game_sample = clean_data[
    (clean_data["num_games"] >= 10) & (clean_data["num_games"] <= 20)
].sample(
    min(
        10,
        len(
            clean_data[
                (clean_data["num_games"] >= 10) & (clean_data["num_games"] <= 20)
            ]
        ),
    ),
    random_state=42,
)
for idx, row in low_game_sample.iterrows():
    adjustment = row["score"] - row["score_original"]
    print(
        f"   Player {row['player_id']:>5} | Opening {row['opening_id']:>4} | Games: {row['num_games']:>3} | "
        f"Opening mean: {row['opening_mean']:.4f} | Original: {row['score_original']:.4f} → Adjusted: {row['score']:.4f} | "
        f"Diff: {adjustment:>+.4f} | Confidence: {row['confidence']:.3f}"
    )

print(f"\n   {'='*120}")
print(f"   Medium-game entries (50-100 games) - MODERATE shrinkage:")
print(f"   {'='*120}")

med_game_sample = clean_data[
    (clean_data["num_games"] >= 50) & (clean_data["num_games"] <= 100)
].sample(
    min(
        10,
        len(
            clean_data[
                (clean_data["num_games"] >= 50) & (clean_data["num_games"] <= 100)
            ]
        ),
    ),
    random_state=42,
)
for idx, row in med_game_sample.iterrows():
    adjustment = row["score"] - row["score_original"]
    print(
        f"   Player {row['player_id']:>5} | Opening {row['opening_id']:>4} | Games: {row['num_games']:>3} | "
        f"Opening mean: {row['opening_mean']:.4f} | Original: {row['score_original']:.4f} → Adjusted: {row['score']:.4f} | "
        f"Diff: {adjustment:>+.4f} | Confidence: {row['confidence']:.3f}"
    )

print(f"\n   {'='*120}")
print(f"   High-game entries (200+ games) - LOW shrinkage:")
print(f"   {'='*120}")

high_game_sample = clean_data[clean_data["num_games"] >= 200].sample(
    min(10, len(clean_data[clean_data["num_games"] >= 200])), random_state=42
)
for idx, row in high_game_sample.iterrows():
    adjustment = row["score"] - row["score_original"]
    print(
        f"   Player {row['player_id']:>5} | Opening {row['opening_id']:>4} | Games: {row['num_games']:>3} | "
        f"Opening mean: {row['opening_mean']:.4f} | Original: {row['score_original']:.4f} → Adjusted: {row['score']:.4f} | "
        f"Diff: {adjustment:>+.4f} | Confidence: {row['confidence']:.3f}"
    )

# Show extreme cases - comparing to both opening mean AND global mean
print(f"\n7️⃣  Extreme cases (showing why opening-specific shrinkage matters):")

# Find entries where opening mean differs significantly from global mean
clean_data["opening_deviation_from_global"] = (
    clean_data["opening_mean"] - global_mean_score
).abs()

print(f"\n   Openings with HIGHEST win rates (strong for White):")
strong_openings = clean_data.nlargest(5, "opening_mean")[
    ["opening_id", "opening_mean", "eco"]
].drop_duplicates("opening_id")
for idx, row in strong_openings.iterrows():
    num_entries = len(clean_data[clean_data["opening_id"] == row["opening_id"]])
    deviation = row["opening_mean"] - global_mean_score
    print(
        f"   Opening {row['opening_id']:>4} ({row['eco']:>3}): mean = {row['opening_mean']:.4f} "
        f"(+{deviation:.4f} vs global) | {num_entries} player entries"
    )

print(f"\n   Openings with LOWEST win rates (weak for White):")
weak_openings = clean_data.nsmallest(5, "opening_mean")[
    ["opening_id", "opening_mean", "eco"]
].drop_duplicates("opening_id")
for idx, row in weak_openings.iterrows():
    num_entries = len(clean_data[clean_data["opening_id"] == row["opening_id"]])
    deviation = row["opening_mean"] - global_mean_score
    print(
        f"   Opening {row['opening_id']:>4} ({row['eco']:>3}): mean = {row['opening_mean']:.4f} "
        f"({deviation:.4f} vs global) | {num_entries} player entries"
    )

# Show specific examples where hierarchical shrinkage made a difference
print(f"\n8️⃣  Examples showing hierarchical shrinkage benefit:")

# Find entries with strong openings where player did well
strong_opening_ids = clean_data.nlargest(50, "opening_mean")["opening_id"].unique()
strong_examples = clean_data[
    (clean_data["opening_id"].isin(strong_opening_ids))
    & (clean_data["num_games"] <= 20)
    & (clean_data["score_original"] > 0.6)
].sample(
    min(
        3,
        len(
            clean_data[
                (clean_data["opening_id"].isin(strong_opening_ids))
                & (clean_data["num_games"] <= 20)
                & (clean_data["score_original"] > 0.6)
            ]
        ),
    ),
    random_state=42,
)

print(
    f"\n   Strong opening + good player performance (shrunk toward HIGH opening mean):"
)
for idx, row in strong_examples.iterrows():
    adjustment = row["score"] - row["score_original"]
    global_shrink_would_be = (
        (row["num_games"] * row["score_original"]) + (K_PLAYER * global_mean_score)
    ) / (row["num_games"] + K_PLAYER)
    difference = row["score"] - global_shrink_would_be
    print(
        f"   Player {row['player_id']:>5} | Opening {row['opening_id']:>4} ({row['eco']:>3}) | Games: {row['num_games']:>2} | "
        f"Opening mean: {row['opening_mean']:.4f} | Original: {row['score_original']:.4f} → {row['score']:.4f}"
    )
    print(
        f"      If we'd shrunk to global mean: {global_shrink_would_be:.4f} (would lose {difference:+.4f} of deserved credit)"
    )

# Find entries with weak openings where player did poorly
weak_opening_ids = clean_data.nsmallest(50, "opening_mean")["opening_id"].unique()
weak_examples = clean_data[
    (clean_data["opening_id"].isin(weak_opening_ids))
    & (clean_data["num_games"] <= 20)
    & (clean_data["score_original"] < 0.45)
].sample(
    min(
        3,
        len(
            clean_data[
                (clean_data["opening_id"].isin(weak_opening_ids))
                & (clean_data["num_games"] <= 20)
                & (clean_data["score_original"] < 0.45)
            ]
        ),
    ),
    random_state=42,
)

print(f"\n   Weak opening + poor player performance (shrunk toward LOW opening mean):")
for idx, row in weak_examples.iterrows():
    adjustment = row["score"] - row["score_original"]
    global_shrink_would_be = (
        (row["num_games"] * row["score_original"]) + (K_PLAYER * global_mean_score)
    ) / (row["num_games"] + K_PLAYER)
    difference = row["score"] - global_shrink_would_be
    print(
        f"   Player {row['player_id']:>5} | Opening {row['opening_id']:>4} ({row['eco']:>3}) | Games: {row['num_games']:>2} | "
        f"Opening mean: {row['opening_mean']:.4f} | Original: {row['score_original']:.4f} → {row['score']:.4f}"
    )
    print(
        f"      If we'd shrunk to global mean: {global_shrink_would_be:.4f} (would unfairly boost by {-difference:+.4f})"
    )

# Drop temporary columns
print(f"\n9️⃣  Cleaning up...")
clean_data = clean_data.drop(
    columns=["score_original", "opening_mean", "opening_deviation_from_global"]
)
print(f"   ✓ Removed temporary columns")

print(f"\n" + "=" * 60)
print("✅ HIERARCHICAL BAYESIAN ADJUSTMENT COMPLETE")
print("=" * 60)
print(f"\nFinal data shape: {clean_data.shape}")
print(f"Columns: {list(clean_data.columns)}")
print(f"\nNew columns added:")
print(f"   • 'confidence': weight for loss function (range [0,1])")
print(f"   • 'score': adjusted using hierarchical Bayesian shrinkage")
print(f"\nKey improvement over simple shrinkage:")
print(f"   • Player scores now shrink toward OPENING-SPECIFIC means, not global mean")
print(f"   • Preserves opening difficulty differences")
print(f"   • More accurate for both strong and weak openings")

STEP 2B: HIERARCHICAL BAYESIAN SCORE ADJUSTMENT

⚙️  Configuration:
   • K_PLAYER (shrinkage constant): 50
   • Method: Two-level empirical Bayes shrinkage
   • Level 1: Calculate opening-specific means
   • Level 2: Shrink player scores toward opening means

📊 Global statistics:
   • Global mean score: 0.5110
   • Total entries: 2,975,888
   • Unique openings: 2,718

1️⃣  LEVEL 1: Calculating opening-specific means...
   ✓ Calculated means for 2,718 openings

   Opening mean score distribution:
   • Min: 0.1667
   • 25th percentile: 0.4961
   • Median: 0.5163
   • 75th percentile: 0.5366
   • Max: 1.0000
   • Std: 0.0504

   Opening sample size distribution:
   • Total games per opening (median): 4913
   • Players per opening (median): 160
   • Total games range: [10, 5648567]
   • Players range: [1, 44105]

2️⃣  LEVEL 2: Shrinking player scores toward opening means...
   Formula: adjusted_score = (num_games × player_score + 50 × opening_mean) / (num_games + 50)
   ✓ Scores adjusted f

In [ ]:
print(clean_data.sample().to_string())

         player_id  opening_id  num_games     score  eco  confidence
2975878      50000        1520         31  0.525188  C20    0.382716
2975879      50000        1642         19  0.532716  C30    0.275362
2975880      50000        1665         42  0.510040  C30    0.456522
2975881      50000        1673         12  0.496546  C31    0.193548
2975882      50000        1740         24  0.542961  C34    0.324324
2975883      50000        1748         15  0.519497  C36    0.230769
2975884      50000        2443         15  0.565022  D00    0.230769
2975885      50000        2668         10  0.507630  D35    0.166667
2975886      50000        3163         12  0.529981  B02    0.193548
2975887      50000        3242         13  0.565552  C34    0.206349
